# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [11]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,
                     header=True)

In [12]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [6]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
from pyspark.ml.feature import VectorAssembler

In [13]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [14]:
output = assembler.transform(data)

In [15]:
final_data = output.select('features','churn')

In [16]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [17]:
from pyspark.ml.classification import LogisticRegression

In [18]:
lr_churn = LogisticRegression(labelCol='churn')

In [19]:
fitted_churn_model = lr_churn.fit(train_churn)

In [20]:
training_sum = fitted_churn_model.summary

In [21]:
training_sum.predictions.describe().show()

+-------+-------------------+------------------+
|summary|              churn|        prediction|
+-------+-------------------+------------------+
|  count|                652|               652|
|   mean|0.15030674846625766|0.1165644171779141|
| stddev| 0.3576462107430231|0.3211469085292243|
|    min|                0.0|               0.0|
|    max|                1.0|               1.0|
+-------+-------------------+------------------+



In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [24]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[5.34471617674081...|[0.99524937041936...|       0.0|
|[27.0,8628.8,1.0,...|    0|[6.33805147453873...|[0.99823537602619...|       0.0|
|[29.0,8688.17,1.0...|    1|[3.01521471812536...|[0.95325676330768...|       0.0|
|[29.0,10203.18,1....|    0|[4.29541261325262...|[0.98655235785918...|       0.0|
|[29.0,13240.01,1....|    0|[7.82368749494204...|[0.99960001661864...|       0.0|
|[29.0,13255.05,1....|    0|[4.68893944983506...|[0.99088736944188...|       0.0|
|[30.0,11575.37,1....|    1|[4.52829586200714...|[0.98931631034759...|       0.0|
|[30.0,13473.35,0....|    0|[2.91049092284842...|[0.94836261079399...|       0.0|
|[31.0,7073.61,0.0...|    0|[3.52704675165173...|[0.97144761156960...|       0.0|
|[31.0,10058.87,

In [25]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                          labelCol='churn')

In [26]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [27]:
auc

0.7264521193092621

In [28]:
final_lr_model = lr_churn.fit(final_data)

In [29]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True,header=True)

In [30]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [31]:
test_new_customers = assembler.transform(new_customers)

In [32]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [33]:
final_results = final_lr_model.transform(test_new_customers)

In [34]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

